# Glucose analysis

* Analysis of continuous glucose monitor data.
* Use `diametrics` python package for analysis (install from source)


#### group assignments

* Athlete
    - M001
    - M002
    - MPS005
    - MPS006
    - MPS007
    - MPS009
* Control
    - MPS008

  

## 1 preprocess data for continuous glucose monitors (CGMs)

* load the datafiles
    - data are stored in directory `raw_data/cgm_data/`
* extract the relevant windows
    * workout
        * workout_start
        * workout_end
    * up to 7 days after workout
* extract the features
    * Mean amplitude of glycemic excursions (MAGE)
    * LBGI
    * HBGI
    * Mean average glucose (`mean_avg_glc`)

      
## 2 plot glucose curves during workout and during day of workout (12am of day of workout until `workout_end + 24 hours`)

* use the `workout_start` and `workout_end` timestamps
* use white background for the workout time window
* use blue background for 10pm to 6am
* use light grey background for other times


## 3 Regression model

* regression of glucose features ~ proteomics proteins
    * MAGE ~ proteins
    * mean_avg_glc ~ proteins

In [ ]:
# prompt: connect to google drive code

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
project_path = '/content/drive/MyDrive/projects/glucose-data-science'
data_path = f'{project_path}/data'

In [ ]:
!cp -r /content/drive/MyDrive/projects/glucose-data-science/notebook/utils.py .

In [ ]:
import os

import datetime as dt
import numpy as np
import pandas as pd

import utils

## Helpers

## 1 preprocess data for continuous glucose monitors (CGMs)

In [ ]:
def preprocess_data(file_name):
  """
  Parameters
  ----------
  file_name: str -

  Returns
  -------
  pd.DataFrame -
  """
  df = pd.read_csv(file_name)
  df['file_name'] = file_name
  df['timestamp'] = pd.to_datetime(df['Timestamp (YYYY-MM-DDThh:mm:ss)'])
  df['duration'] = pd.to_datetime(df['Duration (hh:mm:ss)'])

  return df

### load demographic metadata

In [ ]:
df_list = []

for f in os.listdir(data_path):
  d = pd.read_csv(f'{data_path}/{f}')
  d['file_name'] = f
  d['timestamp'] = pd.to_datetime(d['Timestamp (YYYY-MM-DDThh:mm:ss)'])
  d['duration'] = pd.to_datetime(d['Duration (hh:mm:ss)'])
  df_list.append(d)

len(df_list)

In [ ]:
df = pd.concat(df_list)
df.head()
df.dropna(subset=['Duration (hh:mm:ss)'])

In [ ]:
df[['Duration (hh:mm:ss)']].value_counts()

In [ ]:
link_to_demographic_metadata = f'{project_path}/raw_data/demographic_metadata.csv'
df_metadata = pd.read_csv(link_to_demographic_metadata)

date_format = '%m/%d/%Y %I:%M:%S %p'
df_metadata['workout_start'] = pd.to_datetime(df_metadata['window_start'], format=date_format)
df_metadata['workout_end'] = pd.to_datetime(df_metadata['window_end'], format=date_format)
df_metadata['R-1'] = pd.to_datetime(df_metadata['R-1'], format=date_format)
df_metadata['R'] = pd.to_datetime(df_metadata['R'], format=date_format)
df_metadata['R+1'] = pd.to_datetime(df_metadata['R+1'], format=date_format)
df_metadata['R+7'] = pd.to_datetime(df_metadata['R+7'], format=date_format, errors='coerce')

df_metadata

## load the raw data from `raw_data/cgm_data` using `diametrics` package

## extract relevant windows

## extract features

In [ ]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------------
# 1. LOAD DEMOGRAPHIC METADATA
# ------------------------------------------------------------------------------
metadata_path = r'../../../proteomics_data_analysis_task/raw_data/demographic_metadata.csv'
metadata_df = pd.read_csv(metadata_path)

# Convert relevant columns to datetime
time_cols = ['workout_start', 'workout_end', 'R-1', 'R', 'R+1', 'R+7']
for col in time_cols:
    metadata_df[col] = pd.to_datetime(metadata_df[col])

# ---------------------------------------------------------------------------
# 2. LOAD ALL CGM FILES, EXTRACT SUBJECT ID FROM CELL A2, THEN READ THE DATA
# ---------------------------------------------------------------------------
cgm_folder = r'../../../proteomics_data_analysis_task/raw_data/cgm_data'
# Find all .csv files like M001.csv, MPS005.csv, etc.
cgm_files = glob.glob(os.path.join(cgm_folder, '*.csv'))

cgm_data_dict = {}  # Key = user_id (e.g., "M001"), Value = DataFrame of CGM data

for file_path in cgm_files:
    # Extract the user ID from the filename: "M001.csv" -> "M001"
    base_name = os.path.basename(file_path)     # "M001.csv"
    user_id   = os.path.splitext(base_name)[0]  # "M001"

    # Read the CSV, skipping the first two lines so that
    # the 3rd line is recognized as column headers
    df = pd.read_csv(file_path, skiprows=2)

    # Typical columns might be:
    # "Device", "Serial Number", "Device Timestamp", "Record Type",
    # "Historic Glucose mg/dL", "Scan Glucose mg/dL"
    # Adjust names as needed:
    df.rename(
        columns={
            'Device Timestamp': 'time',
            'Historic Glucose mg/dL': 'glc_hist',
            'Scan Glucose mg/dL': 'glc_scan'
        },
        inplace=True
    )

    # Convert 'time' to datetime
    df['time'] = pd.to_datetime(df['time'])

    # Optionally combine historic + scan glucose into a single 'glc' column
    df['glc'] = df['glc_hist'].combine_first(df['glc_scan'])

    # Keep only the columns you need (time and glc)
    df = df[['time', 'glc']].dropna(subset=['glc']).sort_values('time').reset_index(drop=True)

    # Store in our dictionary using the user_id
    cgm_data_dict[user_id] = df

# ---------------------------------------------------------------------------
# 3. FOR EACH SUBJECT IN METADATA: FILTER CGM DATA AND PLOT WINDOWS
# ---------------------------------------------------------------------------
for idx, meta_row in metadata_df.iterrows():
    sub_id = meta_row['subject_id']  # e.g., "M001" or "MPS005"
    group  = meta_row.get('group', '')  # If you have a 'group' column

    # Check if we have CGM data for this subject ID
    if sub_id not in cgm_data_dict:
        print(f"No CGM data found for subject {sub_id}")
        continue

    # Retrieve CGM data from our dictionary
    subject_df = cgm_data_dict[sub_id].copy()

    # Extract relevant timestamps (R-1, R, R+1, R+7)
    t_r_minus_1 = meta_row.get('R-1', None)
    t_r         = meta_row.get('R',   None)
    t_r_plus_1  = meta_row.get('R+1', None)
    t_r_plus_7  = meta_row.get('R+7', None)

    # Create a multi-subplot figure to illustrate each window
    plt.figure(figsize=(10, 6))
    plt.suptitle(f'Glucose Levels for {sub_id} (Group: {group})', fontsize=14)

    def plot_window(ax, start_time, end_time, label):
        # Filter subject_df by the given time window
        if pd.notnull(start_time) and pd.notnull(end_time):
            mask = (subject_df['time'] >= start_time) & (subject_df['time'] <= end_time)
            window_df = subject_df.loc[mask]

            ax.plot(window_df['time'], window_df['glc'], marker='o', label=label)
            ax.set_title(label)
            ax.set_xlabel('Time')
            ax.set_ylabel('Glucose (mg/dL)')
            ax.tick_params(axis='x', rotation=45)
            ax.legend()

    # Subplot 1: R-1 to R
    ax1 = plt.subplot(2, 2, 1)
    plot_window(ax1, t_r_minus_1, t_r, 'R-1 → R')

    # Subplot 2: R to R+1
    ax2 = plt.subplot(2, 2, 2)
    plot_window(ax2, t_r, t_r_plus_1, 'R → R+1')

    # Subplot 3: R to R+7
    ax3 = plt.subplot(2, 2, 3)
    plot_window(ax3, t_r, t_r_plus_7, 'R → R+7')

    # Adjust layout and show
    plt.tight_layout()
    plt.show()
    # Or save each subject's figure:
    # plt.savefig(f'{sub_id}_glucose_windows.png', dpi=150)

In [ ]:
#####
# Concatenate glucose data vertically
#####

# Assuming `cgm_data_dict` is defined
# Create an empty list to store dataframes
df_list = []

# Iterate through the dictionary and append ID to each dataframe
for key, df in cgm_data_dict.items():
    df["subject_id"] = key  # Add a column with the key as the ID
    df_list.append(df)  # Append to the list

# Concatenate all dataframes
combined_df_cgm = pd.concat(df_list, axis=0)

# Reset index for the combined dataframe
combined_df_cgm.reset_index(drop=True, inplace=True)

# Display the resulting dataframe
print(combined_df_cgm)

In [ ]:
plt.savefig(f'{sub_id}_glucose_windows.png', dpi=150)

In [ ]:
#############################################
# NOTEBOOK 3 EXAMPLE CODE
#############################################

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
from datetime import timedelta

# If diametrics is not in PYTHONPATH, you might need a local install or sys.path
# from diametrics import metrics, preprocessing

#############################################
# 1) LOAD PROTEOMICS DATA
#############################################

proteomics_path = r"../../../proteomics_data_analysis_task/raw_data/proteomics_data/df_data_proteomics_olink_running_proteins_plate2a.csv"
df_prot = pd.read_csv(proteomics_path)

print("Proteomics data loaded. Columns:", df_prot.columns.to_list())
print(df_prot.head(3))

# We assume this has columns like:
#   subject_id, group, timepoint, Assay, NPX
# Possibly 1070 unique Assay values, timepoints in {R-1, R, R+1, R+7}, etc.

#############################################
# 2) LOAD CGM DATA + WORKOUT METADATA
#############################################

#cgm_path = "cgm_data.csv"
#df_cgm = pd.read_csv(cgm_path)
df_cgm = combined_df_cgm
df_cgm['time'] = pd.to_datetime(df_cgm['time'])

meta_path = "../../../proteomics_data_analysis_task/raw_data/demographic_metadata.csv"
df_meta = pd.read_csv(meta_path)
df_meta['workout_start'] = pd.to_datetime(df_meta['workout_start'])
df_meta['workout_end']   = pd.to_datetime(df_meta['workout_end'])

print("\nCGM data columns:", df_cgm.columns)
print("Metadata columns:", df_meta.columns)

#############################################
# 3) EXTRACT ~36 HRS OF CGM & PLOT WITH SHADING
#############################################

def extract_cgm_window(df, start_dt, hours=36):
    """
    Takes a CGM DataFrame (df) with a 'time' column,
    extracts from 'start_dt' to start_dt + 'hours' hours.
    """
    end_dt = start_dt + timedelta(hours=hours)
    subset = df[(df['time'] >= start_dt) & (df['time'] <= end_dt)].copy()
    return subset

def plot_glucose_trace(df, workout_start, workout_end, subject_id=None):
    """
    Plot time vs. glc from df, shading workout & sleep intervals.
    subject_id is optional, just for the title/label if needed.
    """
    fig, ax = plt.subplots(figsize=(10,4))

    ax.plot(df['time'], df['glc'], color='blue', label='Glucose Trace')

    # Shade workout
    ax.axvspan(workout_start, workout_end, alpha=0.3, color='red', label='Workout')

    # Assume typical sleep: 22:00 -> 06:00
    # We'll do it for the first night only (if needed).
    day_of_workout = workout_start.replace(hour=0, minute=0, second=0, microsecond=0)
    sleep_start = day_of_workout + pd.Timedelta(hours=22)
    sleep_end   = day_of_workout + pd.Timedelta(hours=30)  # next day 6am
    ax.axvspan(sleep_start, sleep_end, alpha=0.1, color='gray', label='Sleep')

    title_txt = f"Glucose Trace (Subject: {subject_id})" if subject_id else "Glucose Trace"
    ax.set_title(title_txt, fontsize=14)
    ax.set_xlabel("Time", fontsize=12)
    ax.set_ylabel("Glucose", fontsize=12)
    ax.legend()
    plt.tight_layout()
    plt.show()

#############################################
# 4) COMPUTE CGM METRICS (EXAMPLE USING DIAMETRICS)
#############################################

import sys
sys.path.append("./diametrics/")
from src.diametrics import transform, metrics, preprocessing, visualizations


def compute_glycemic_metrics(df):
    """
    Example wrapper: if you're using diametrics,
    df should have columns: time, glc, maybe subject_id.
    """
    # If needed, detect or change units
    units = preprocessing.detect_units(df)
    df = preprocessing.change_units(df)  # if you must unify mg/dL vs. mmol

    # We'll do an example call to all_standard_metrics
    # This returns a DF with various columns (start_dt, end_dt, avg_glc, etc.)
    # if 'ID' not in df.columns, we can treat as single subject by ignoring grouping
    # or set df['ID'] = 'single_subject'
    # df['ID'] = df.get('ID', 'dummy_subject')

    cgm_metrics_df = metrics.all_standard_metrics(df, units=None, gap_size=5)
    return cgm_metrics_df

    # For demonstration if diametrics not installed, return a dummy DF:
    return pd.DataFrame({
        'subject_id': [df['subject_id'].iloc[0] if 'subject_id' in df.columns else 'unknown'],
        'avg_glc': [df['glc'].mean()],
        'mage': [df['glc'].std()*1.5],  # not real MAGE calc, just a placeholder
        'time_in_range': [80.0]        # dummy
    })

#############################################
# 5) MERGE PROTEOMICS + CGM METRICS + DO REGRESSION
#############################################

from sklearn.linear_model import LinearRegression

def regression_proteins_to_cgm(df_proteomics, df_cgm_metrics):
    """
    Example:
      - df_proteomics has columns like [subject_id, group, timepoint, Assay, NPX].
      - df_cgm_metrics has [subject_id, avg_glc, mage, time_in_range, etc.].
    We'll do a simple pivot on df_proteomics to get each subject's average NPX
    for each protein or timepoint, then merge with df_cgm_metrics on subject_id.
    Then run a LinearRegression for demonstration.
    """
    # 5a) Pivot proteomics: subject_id => row, (Assay) => columns
    # We'll pick a single timepoint, or average across R-1, R, ...
    # For demonstration, let's pick R-1 (pre-workout) as the features
    prot_rminus1 = df_proteomics[df_proteomics['timepoint'] == 'R-1'].copy()
    if prot_rminus1.empty:
        print("No data at R-1 for proteomics. Skipping regression.")
        return

    pivot_df = prot_rminus1.pivot_table(
        index='subject_id',
        columns='Assay',
        values='NPX',
        aggfunc='mean'  # if multiple rows per subject/assay
    ).reset_index()

    # 5b) Merge with cgm_metrics on subject_id
    merged = pd.merge(pivot_df, df_cgm_metrics, on='subject_id', how='inner')
    print("Merged shape:", merged.shape)

    # Suppose we want to predict avg_glc from all protein columns
    # We'll find columns that are the proteomics assays
    protein_cols = prot_rminus1['Assay'].unique().tolist()
    # remove any that appear in df_cgm_metrics
    Xcols = [col for col in protein_cols if col in merged.columns]

    # define X and y
    X = merged[Xcols].fillna(0).values  # naive fill for missing
    y = merged['mage'].values # replace with column for other features as desired

    if len(X) < 2:
        print("Not enough data for regression.")
        return

    model = LinearRegression()
    model.fit(X, y)
    r2 = model.score(X, y)
    print("LinearRegression R^2 for avg_glc:", r2)

    # Show top coefficients (if many proteins, this is large)
    for protein, coef in zip(Xcols, model.coef_):
        print(f"{protein}: {coef:.3f}")

    df_protein_coef = pd.DataFrame([])
    df_protein_coef['Protein'] = Xcols
    df_protein_coef['coef'] = model.coef_

    model_intercept = model.intercept_

    return model, df_protein_coef

#############################################
# MAIN LOGIC: DEMONSTRATION
#############################################
if __name__ == "__main__":
    cgm_metrics_storage = []

    # Loop over each subject in metadata
    for idx, row in df_meta.iterrows():
        subj_id = row['subject_id']
        wstart = row['workout_start']
        wend   = row['workout_end']

        # Extract that subject's CGM from df_cgm
        cgm_subj = df_cgm[df_cgm['subject_id'] == subj_id].copy()
        if cgm_subj.empty:
            print(f"No CGM data for subject {subj_id}")
            continue

        # We'll define the start for the 36-hour window as midnight of workout day
        midn_before = wstart.replace(hour=0, minute=0, second=0, microsecond=0)
        cgm_window = extract_cgm_window(cgm_subj, start_dt=midn_before, hours=36)
        if cgm_window.empty:
            print(f"No CGM readings in the 36-hr window for {subj_id}")
            continue

        # Plot
        plot_glucose_trace(cgm_window, wstart, wend, subject_id=subj_id)

        # Compute CGM metrics (example)
        # Optionally add 'subject_id' column so metrics can be merged
        cgm_window['subject_id'] = subj_id
        metrics_df = compute_glycemic_metrics(cgm_window)
        metrics_df['subject_id']=subj_id
        print(f"Computed metrics for subj {subj_id}:\n", metrics_df)

        # Collect results in a list to merge later
        cgm_metrics_storage.append(metrics_df)

    # Combine all subjects' CGM metrics
    if len(cgm_metrics_storage) > 0:
        cgm_all_metrics = pd.concat(cgm_metrics_storage, ignore_index=True)
    else:
        cgm_all_metrics = pd.DataFrame()

    print("\nAll subject CGM metrics:\n", cgm_all_metrics)

    # Finally, run regression: proteomics -> CGM metrics
    # We'll use our df_prot (proteomics) and the cgm_all_metrics
    if not cgm_all_metrics.empty:
        model, df_protein_coefs = regression_proteins_to_cgm(df_prot, cgm_all_metrics)
        # This tries to predict 'avg_glc' from NPX (R-1) in proteomics
    else:
        print("No CGM metrics available, skipping regression.")


In [ ]:
df_protein_coefs

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df_protein_coefs is already defined
# Sort dataframe by 'coef' in descending order
df_sorted = df_protein_coefs.sort_values(by="coef", ascending=False)

# Create the horizontal bar chart
plt.figure(figsize=(12, 150))
plt.barh(df_sorted["Protein"], df_sorted["coef"], color='blue', alpha=0.7)
plt.ylabel("Protein", fontsize=12)
plt.xlabel("Coefficient", fontsize=12)
plt.title("Protein Coefficients Sorted by Largest Value", fontsize=14)
plt.tight_layout()
plt.show()